In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import pandas as pd
import numpy as np

In [4]:
#database connection settings
import psycopg2

db_name = "traviato_development"
db_host = "localhost"
db_port = "5432"
db_user = "lievgarcia"
db_pwd = "traviato81"

conn = psycopg2.connect(database=db_name, user=db_user, password=db_pwd, host=db_host, port=db_port)

In [5]:
### COMPOSERS AKA UPDATE
from bs4 import BeautifulSoup
import requests, json
headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0' }

cursor = conn.cursor()  
cursor.execute("SELECT distinct name, url, uri FROM composers c WHERE aka IS NULL")
comp_df = pd.DataFrame(cursor.fetchall(), columns=['name', 'url', 'uri'])

for idx,row in comp_df.iterrows():
    
    wikitext = ''
    aliases = []    
    scrape = True
    
    url = row['url']
    r  = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, "html5lib")    
    try:
        wikidata = json.loads(str(soup.body.text))
        uri = list(wikidata['entities'].keys())[0]
    except:
        scrape = None
    
    #getting all aliases, also known as for item
    try:
        if scrape:
            for k in wikidata['entities'][uri]['aliases']:
    #             print(wikidata['entities'][uri]['aliases'][k][0]['value'])
                aliases.append(wikidata['entities'][uri]['aliases'][k][0]['value'])
            aliases.append(row['name'])
            aliases = '; '.join(aliases)
            update = "UPDATE COMPOSERS SET aka = '" + aliases.replace("'", "''") + "' WHERE uri = '" + uri + "'; COMMIT;"            
            cursor.execute(update)
#             print(update)
    except:
        None

In [82]:
### COMPOSERS WIKIPEDIA TEXT UPDATE
from bs4 import BeautifulSoup
import requests, json
headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0' }
        
cursor = conn.cursor()  
cursor.execute("SELECT distinct name, url, uri FROM composers c WHERE wikipedia_text IS NULL")
comp_df = pd.DataFrame(cursor.fetchall(), columns=['name', 'url', 'uri'])

for idx,row in comp_df.iterrows():
    
    wikitext = ''
    aliases = []    
    scrape = True
    
    url = row['url']
    r  = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, "html5lib")    
    try:
        wikidata = json.loads(str(soup.body.text))
        uri = list(wikidata['entities'].keys())[0]
    except:
        scrape = None
    
    #getting all wiki text on their page, also known as for item        
    try:
        if scrape:
            wiki_url = wikidata['entities'][uri]['sitelinks']['ruwiki']['url']
            r_wiki  = requests.get(wiki_url, headers=headers)
            soup = BeautifulSoup(r_wiki.text, "html5lib")    
            wikitext = soup.body.get_text()      
            update = "UPDATE COMPOSERS SET wikipedia_text = '" + wikitext.replace("'", "''") + "' WHERE uri = '" + uri + "'; COMMIT;"            
            cursor.execute(update)            
    except: 
        None

In [22]:
#database connection settings
import psycopg2

db_name = "traviato_development"
db_host = "localhost"
db_port = "5432"
db_user = "lievgarcia"
db_pwd = "traviato81"

conn = psycopg2.connect(database=db_name, user=db_user, password=db_pwd, host=db_host, port=db_port)

dict_keys(['pageid', 'ns', 'title', 'lastrevid', 'modified', 'type', 'id', 'labels', 'descriptions', 'aliases', 'claims', 'sitelinks'])

In [6]:
### WORKS AKA UPDATE
from bs4 import BeautifulSoup
import requests, json
headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0' }

cursor = conn.cursor()  
cursor.execute("SELECT distinct name, url, uri FROM works c WHERE aka IS NULL")
work_df = pd.DataFrame(cursor.fetchall(), columns=['name', 'url', 'uri'])

for idx,row in work_df.iterrows():
    
    wikitext = ''
    aliases = []    
    scrape = True
    
    url = row['url']
    r  = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, "html5lib")    
    try:
        wikidata = json.loads(str(soup.body.text))
        uri = list(wikidata['entities'].keys())[0]
    except:
        scrape = None
    
    #getting all aliases, also known as for item
    try:
        if scrape:
            for k in wikidata['entities'][uri]['aliases']:
    #             print(wikidata['entities'][uri]['aliases'][k][0]['value'])
                aliases.append(wikidata['entities'][uri]['aliases'][k][0]['value'])
            aliases.append(row['name'])        
            aliases = '; '.join(aliases)
            update = "UPDATE WORKS SET aka = '" + aliases.replace("'", "''") + "' WHERE uri = '" + uri + "'; COMMIT;"            
            cursor.execute(update)
#             print(update)
    except:
        None

In [27]:
### WORKS WIKIPEDIA TEXT UPDATE
from bs4 import BeautifulSoup
import requests, json
headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0' }
        
# languages = ['nlwiki', 'svwiki', 'plwiki', 'ruwiki']  
# for language in languages:
cursor = conn.cursor()  
cursor.execute("SELECT distinct name, url, uri FROM works c WHERE wikipedia_text IS NULL")
work_df = pd.DataFrame(cursor.fetchall(), columns=['name', 'url', 'uri'])

counters = {}

for idx,row in work_df.iterrows():

    wikitext = ''
    aliases = []    
    scrape = True

    url = row['url']
    r  = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, "html5lib")    
    try:
        wikidata = json.loads(str(soup.body.text))
        uri = list(wikidata['entities'].keys())[0]
#         print(uri)
    except:
        scrape = None

    #getting all wiki text on their page, also known as for item        
    try:
        if scrape:
            wiki_url = wikidata['entities'][uri]['sitelinks']['enwiki']['url']
            r_wiki  = requests.get(wiki_url, headers=headers)
            soup = BeautifulSoup(r_wiki.text, "html5lib")    
            wikitext = soup.body.get_text()      
            update = "UPDATE WORKS SET wikipedia_text = '" + wikitext.replace("'", "''") + "' WHERE uri = '" + uri + "'; COMMIT;"            
            cursor.execute(update)            
    except: 
        try:
#             print('im here')
#             print(wikidata['entities'][uri])
            if wikidata['entities'][uri]['sitelinks']:
                for lg in list(wikidata['entities'][uri]['sitelinks'].keys()):
                    counter = counters.get(lg, 0) + 1
                    counters[value] = counter        
            else:
                None #print('No links')
#                 print(counter)
#             print(wikidata['entities'][uri]['sitelinks'].keys())        
#                 None
        except:
            print(wikidata['entities'][uri])        
#             None
    #         None

{'pageid': 18653509, 'ns': 0, 'title': 'Q17046450', 'lastrevid': 676593880, 'modified': '2018-05-09T16:38:35Z', 'type': 'item', 'id': 'Q17046450', 'labels': {'be': {'language': 'be', 'value': 'Алеся'}, 'ru': {'language': 'ru', 'value': 'Алеся'}, 'uk': {'language': 'uk', 'value': 'Олеся'}}, 'descriptions': {'ru': {'language': 'ru', 'value': 'опера'}, 'be': {'language': 'be', 'value': 'опера'}, 'uk': {'language': 'uk', 'value': 'опера'}}, 'aliases': {'ru': [{'language': 'ru', 'value': 'Девушка с Полесья'}], 'uk': [{'language': 'uk', 'value': 'Дівчина з Полісся'}]}, 'claims': {'P31': [{'mainsnak': {'snaktype': 'value', 'property': 'P31', 'datavalue': {'value': {'entity-type': 'item', 'numeric-id': 1344, 'id': 'Q1344'}, 'type': 'wikibase-entityid'}, 'datatype': 'wikibase-item'}, 'type': 'statement', 'id': 'Q17046450$46C9128B-8ACC-4F69-8CC5-A9604075ABEA', 'rank': 'normal'}], 'P86': [{'mainsnak': {'snaktype': 'value', 'property': 'P86', 'datavalue': {'value': {'entity-type': 'item', 'numeric

KeyboardInterrupt: 

In [24]:
counters

{}